In [1]:
import yfinance as yf
import pandas as pd

ticker = yf.Ticker("TD")
stock_data = ticker.history(period="1y")
stock_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-11-02 00:00:00-04:00,53.897357,55.333984,53.887841,55.257874,3390700,0.0,0.0
2023-11-03 00:00:00-04:00,56.019002,56.675473,55.800178,56.351994,6883900,0.0,0.0
2023-11-06 00:00:00-05:00,56.580334,56.989440,56.085600,56.504219,2075700,0.0,0.0
2023-11-07 00:00:00-05:00,56.104627,56.152196,55.647950,55.933372,1595100,0.0,0.0
2023-11-08 00:00:00-05:00,55.819202,56.218795,55.543295,55.838230,1608000,0.0,0.0
...,...,...,...,...,...,...,...
2024-10-28 00:00:00-04:00,56.419998,56.470001,56.040001,56.279999,2710300,0.0,0.0
2024-10-29 00:00:00-04:00,56.200001,56.200001,55.230000,55.480000,3023300,0.0,0.0
2024-10-30 00:00:00-04:00,55.330002,55.770000,55.220001,55.650002,3607300,0.0,0.0


In [2]:
df = stock_data.reset_index(drop=False)
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-11-02 00:00:00-04:00,53.897357,55.333984,53.887841,55.257874,3390700,0.0,0.0
1,2023-11-03 00:00:00-04:00,56.019002,56.675473,55.800178,56.351994,6883900,0.0,0.0
2,2023-11-06 00:00:00-05:00,56.580334,56.989440,56.085600,56.504219,2075700,0.0,0.0
3,2023-11-07 00:00:00-05:00,56.104627,56.152196,55.647950,55.933372,1595100,0.0,0.0
4,2023-11-08 00:00:00-05:00,55.819202,56.218795,55.543295,55.838230,1608000,0.0,0.0
...,...,...,...,...,...,...,...,...
247,2024-10-28 00:00:00-04:00,56.419998,56.470001,56.040001,56.279999,2710300,0.0,0.0
248,2024-10-29 00:00:00-04:00,56.200001,56.200001,55.230000,55.480000,3023300,0.0,0.0
249,2024-10-30 00:00:00-04:00,55.330002,55.770000,55.220001,55.650002,3607300,0.0,0.0
250,2024-10-31 00:00:00-04:00,55.639999,55.680000,54.939999,55.270000,3488200,0.0,0.0


In [3]:
def trade_decision(prev_price, curr_price, threshold=0.05):
    delta = curr_price - prev_price
    prev_pct_change = delta / prev_price
    curr_pct_change = delta / curr_price
    pct_change = max(prev_pct_change, curr_pct_change)
    if pct_change > 0 and pct_change >= threshold:
        return 'BUY'
    elif pct_change <= 0 and -1 * pct_change >= threshold:
        return 'SELL'
    else:
        return 'HOLD'
    
def trade(prev_price, curr_price, curr_cash, curr_shares, threshold):
    decision = trade_decision(prev_price, curr_price, threshold)

    if decision == 'BUY':
        value = curr_cash * threshold
        share_amount = value / curr_price
        new_share = curr_shares + share_amount
        new_cash = curr_cash - value

    elif decision == 'SELL':
        value = curr_shares * curr_price * threshold
        share_amount = value / curr_price
        new_share = curr_shares - share_amount
        new_cash = curr_cash + value

    else:
        new_cash = curr_cash
        new_share = curr_shares

    return new_cash, new_share

In [4]:
# Initialize account
df['curr_cash'] = 10000
df['curr_shares'] = 0

In [5]:
# Trade daily
for i in range(len(df)):
    if i > 0:
        curr_row = df.iloc[i]
        prev_row = df.iloc[i-1]
        curr_cash, curr_shares = trade(
            prev_price=prev_row['Open'],
            curr_price=curr_row['Open'],
            curr_cash=prev_row['curr_cash'],
            curr_shares=prev_row['curr_shares'],
            threshold=0.02
        )
        df['curr_cash'][i] = curr_cash
        df['curr_shares'][i] = curr_shares

/tmp/ipykernel_8532/271401133.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['curr_cash'][i] = curr_cash
/tmp/ipykernel_8532/271401133.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['curr_shares'][i] = curr_shares


In [6]:
df['total_value'] = df['curr_cash'] + df['curr_shares'] * df['Close']
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,curr_cash,curr_shares,total_value
0,2023-11-02 00:00:00-04:00,53.897357,55.333984,53.887841,55.257874,3390700,0.0,0.0,10000.00000,0.000000,10000.000000
1,2023-11-03 00:00:00-04:00,56.019002,56.675473,55.800178,56.351994,6883900,0.0,0.0,9800.00000,3.570217,10001.188853
2,2023-11-06 00:00:00-05:00,56.580334,56.989440,56.085600,56.504219,2075700,0.0,0.0,9800.00000,3.570217,10001.732332
3,2023-11-07 00:00:00-05:00,56.104627,56.152196,55.647950,55.933372,1595100,0.0,0.0,9800.00000,3.570217,9999.694285
4,2023-11-08 00:00:00-05:00,55.819202,56.218795,55.543295,55.838230,1608000,0.0,0.0,9800.00000,3.570217,9999.354606
...,...,...,...,...,...,...,...,...,...,...,...
247,2024-10-28 00:00:00-04:00,56.419998,56.470001,56.040001,56.279999,2710300,0.0,0.0,8856.51147,19.410678,9948.944429
248,2024-10-29 00:00:00-04:00,56.200001,56.200001,55.230000,55.480000,3023300,0.0,0.0,8856.51147,19.410678,9933.415901
249,2024-10-30 00:00:00-04:00,55.330002,55.770000,55.220001,55.650002,3607300,0.0,0.0,8856.51147,19.410678,9936.715754
250,2024-10-31 00:00:00-04:00,55.639999,55.680000,54.939999,55.270000,3488200,0.0,0.0,8856.51147,19.410678,9929.339676
